In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import seaborn as sns

In [11]:
from sklearn.metrics import mean_squared_error as mse

In [2]:
data=pd.read_csv("C:/Users/91939/Downloads/jharkhand.csv")

In [3]:
data['Timestamp']=pd.to_datetime(data.Timestamp)

In [4]:
data.head()

,Timestamp,demand
0,2022-04-01 00:00:00,63.1060
1,2022-04-01 00:15:00,67.4580
2,2022-04-01 00:30:00,52.4316
3,2022-04-01 00:45:00,54.9388
4,2022-04-01 01:00:00,57.2732


In [6]:
df=data.loc[:479]

In [7]:
df1=df['demand']
dataset=df1.values
dataset=dataset.astype('float32')

In [8]:
train ,test=df.loc[:383], df.loc[384:]
train_size=384
test_size=96

In [36]:
scaler=MinMaxScaler(feature_range=(0,1))


In [37]:
train['demand']=scaler.fit_transform(train[['demand']])


C:\Users\91939\AppData\Local\Temp\ipykernel_24316\2245945729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['demand']=scaler.fit_transform(train[['demand']])


In [38]:
train

,Timestamp,demand
0,2022-04-01 00:00:00,0.586769
1,2022-04-01 00:15:00,0.706947
2,2022-04-01 00:30:00,0.292004
3,2022-04-01 00:45:00,0.361238
4,2022-04-01 01:00:00,0.425701
...,...,...
379,2022-04-04 22:45:00,0.772249
380,2022-04-04 23:00:00,0.247799
381,2022-04-04 23:15:00,0.570002
382,2022-04-04 23:30:00,0.682149


In [39]:
test['demand']=scaler.fit_transform(test[['demand']])

C:\Users\91939\AppData\Local\Temp\ipykernel_24316\2619846879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['demand']=scaler.fit_transform(test[['demand']])


In [40]:
seq_size=96

In [49]:
def to_sequences(x,y,seq_size):
    x_values=[]
    y_values=[]
    for i in range(len(x)-seq_size):
        x_values.append(x.iloc[i:(i+seq_size)].values)
        y_values.append(y.iloc[(i+seq_size)])        
    return np.array(x_values),np.array(y_values)

In [50]:
trainX,trainY=to_sequences(train[['demand']],train['demand'],seq_size)
testX,testY=to_sequences(test[['demand']],test['demand'],seq_size)

In [51]:
testX

array([], dtype=float64)

In [17]:
model = Sequential()
model.add(LSTM(64, input_shape=(trainX.shape[1],trainX.shape[2])))
model.add(Dense(32))
model.add(Dense(1))

#model.add(RepeatVector(trainX.shape[1]))

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________


In [18]:
trainX=np.reshape(trainX,(trainX.shape[0],1,trainX.shape[1]))
testX=np.reshape(testX,(testX.shape[0],1,testX.shape[1]))

IndexError: tuple index out of range

In [ ]:
model.fit(trainX,trainY,validation_data=(testX,testY),verbose=2,epochs=100 )

In [ ]:
trainPredict=model.predict(trainX)
testPredict=model.predict(testX)

In [12]:
def prediction_plot(model,X,y,start=0,end=95):
    predictions=model.predict(X).flatten
    df=pd.DataFrame(data={'predicitions':predictions,'actual':y})
    plt.plot(df['predictions'][start:end])
    return df,mse(y,predictions)